# Analysis of Euros for Doctors workshop at DataHarvest 2017

Run the load_data notebook before to get the data.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from ipywidgets import interact

from eurosfordoctors import checks
from eurosfordoctors import utils


MONEY_FIELDS = utils.MONEY_FIELDS
MONEY_FIELDS_ONLY = utils.MONEY_FIELDS_ONLY
DOC_MONEY_FIELDS = list(set(MONEY_FIELDS_ONLY) - {'sponsorship', 'donations_grants'})
MONEY_FIELDS_DIRTY = ['%s_dirty' % x for x in MONEY_FIELDS_ONLY]

pd.set_option('display.max_rows', 999)
pd.set_option('display.float_format', lambda x: '{0:,.2f}'.format(x))

CURRENCY = 'PLN'

In [2]:
df = pd.read_csv('data/geocoded.csv', converters={'postcode': str, 'uci': str}, encoding='utf-8')
df.head()

,index,address,base_country,clean_name,company,company_doc_id,country,currency,donations_grants,donations_grants_dirty,...,travel_accommodation,travel_accommodation_dirty,type,uci,uid,year,computed_total,lat,lng,uid_original
0,0,al.Wojska Polskiego 30,PL,tomasz-aleksiejczyk,abbvie,NaN,PL,NaN,nan,nan,...,nan,nan,hcp,,659935bb-7b00-4bba-9be2-22a36b764ace,2015,"2,600.00",53.80,20.48,659935bb-7b00-4bba-9be2-22a36b764ace
1,1,ul.Szpitalna 27/33,PL,alicja-bartkowska-sniatkowska,abbvie,NaN,PL,NaN,nan,nan,...,nan,nan,hcp,,72da1abc-64bf-4750-b09b-bd1d6db01625,2015,"1,849.00",52.41,16.88,72da1abc-64bf-4750-b09b-bd1d6db01625
2,2,ul.Roentgena 5,PL,witold-bartnik,abbvie,NaN,PL,NaN,nan,nan,...,nan,nan,hcp,,d0af9ef0-d840-40f2-a2b3-076993d5fe58,2015,"4,030.00",52.15,21.03,d0af9ef0-d840-40f2-a2b3-076993d5fe58
3,3,ul.Zjednoczenia 10,PL,marek-beniowski,abbvie,NaN,PL,NaN,nan,nan,...,"9,517.00","9,517.00",hcp,,0435e01e-af47-4c69-b29c-f917cc012285,2015,"13,272.00",50.29,18.95,0435e01e-af47-4c69-b29c-f917cc012285
4,4,ul. 3 Maja 13/15,PL,szymon-bialka,abbvie,NaN,PL,NaN,nan,nan,...,154.00,154.00,hcp,,e090df0e-5669-435e-bd15-5a1ae474fbad,2015,154.00,50.30,18.79,e090df0e-5669-435e-bd15-5a1ae474fbad


In [3]:
len(df)

394

In [4]:
df[MONEY_FIELDS_ONLY] = df[MONEY_FIELDS_ONLY].applymap(lambda x: x if pd.isnull(x) else (np.nan if x == 0.0 else x))
df['computed_total'] = df[MONEY_FIELDS_ONLY].sum(1)

In [5]:
company_df = pd.read_csv('data/pl/companies.csv')
company_df = company_df[company_df['amount_rd'].notnull()]
company_df = company_df.rename(columns={'amount_rd': 'amount', 'slug': 'company'})[['company', 'amount']]
company_df['amount'] = pd.to_numeric(company_df['amount'])
company_df.head()

,company,amount
0,abbvie,"13,023,266.00"
1,bayer,"11,203,961.44"


In [6]:
agg_df = pd.read_csv('data/pl/aggregated.csv').rename(columns={'slug': 'company'})
agg_df.head()

,company,year,type,label,count,amount,percent
0,abbvie,2015,hcp,registration_fees,757,"1,077,571.00",96.00
1,abbvie,2015,hcp,travel_accommodation,891,"2,323,174.00",94.00
2,abbvie,2015,hcp,fees,321,"956,372.00",93.00
3,abbvie,2015,hcp,related_expenses,42,"68,248.00",95.00
4,abbvie,2015,hco,donations_grants,0,0.00,0.00


## Questions

### 1: How many doctors have received money?

In [7]:
mean_labels_per_doc = df[df['type'] == 'hcp'].groupby(['company', 'uid']).apply(lambda x: x[MONEY_FIELDS_ONLY].count().sum()).unstack().mean(1)

In [8]:
mean_labels_per_doc

company
abbvie   1.41
bayer    1.42
dtype: float64

In [9]:
mean_labels_per_org = df[df['type'] == 'hco'].groupby(['company', 'uid']).apply(lambda x: x[MONEY_FIELDS_ONLY].count().sum()).unstack().mean(1)

In [10]:
individual_count = len(df[df['type'] == 'hcp'].groupby('uid'))
individual_org_count = len(df[df['type'] == 'hco'].groupby('uid'))
print('Named HCP', individual_count)
print('Named HCO', individual_org_count)

Named HCP 145
Named HCO 241


In [11]:
summed_agg_docs = agg_df[agg_df['type'] == 'hcp'].groupby(['company'])['count'].sum()

In [12]:
avg_mean_labels_per_doc = mean_labels_per_doc.mean()
avg_mean_labels_per_doc

1.4183712121212122

In [13]:
avg_mean_labels_per_org = mean_labels_per_org.mean()

In [14]:
estimated_agg_docs_per_company_df = pd.merge(mean_labels_per_doc.to_frame(), summed_agg_docs.to_frame(), how='outer',
                                             left_index=True, right_index=True)

estimated_agg_docs_per_company_df[0] = estimated_agg_docs_per_company_df[0].fillna(avg_mean_labels_per_doc)
estimated_agg_docs_per_company_df['estimate'] = estimated_agg_docs_per_company_df['count'] / estimated_agg_docs_per_company_df[0]
estimated_agg_docs_per_company_df

,0,count,estimate
company,,,
abbvie,1.41,2011,"1,423.72"
bayer,1.42,2175,"1,527.13"


In [15]:
estimated_agg_docs_per_company_df['count'].sum()

4186

In [16]:
estimated_agg_docs_per_company_df[0].mean()

1.4183712121212122

In [17]:
summed_agg_orgs = agg_df[agg_df['type'] == 'hco'].groupby(['company'])['count'].sum()

estimated_agg_orgs_per_company_df = pd.merge(mean_labels_per_org.to_frame(), summed_agg_orgs.to_frame(), how='outer',
                                             left_index=True, right_index=True)
estimated_agg_orgs_per_company_df
estimated_agg_orgs_per_company_df[0] = estimated_agg_orgs_per_company_df[0].fillna(avg_mean_labels_per_org)
estimated_agg_orgs_per_company_df['estimate'] = estimated_agg_orgs_per_company_df['count'] / estimated_agg_orgs_per_company_df[0]


In [18]:
top_docs_by_company_count = df[df['type'] == 'hcp'].groupby(['uid', 'company']).size().unstack().sum(1)
average_company_count_for_doc = top_docs_by_company_count.mean()
average_company_count_for_doc

1.0068965517241379

In [19]:
average_company_count_for_org = df[df['type'] == 'hco'].groupby(['uid', 'company']).size().unstack().sum(1).mean()
average_company_count_for_org

1.0290456431535269

In [20]:
estimated_agg_docs_per_company_df['estimate'].sum()

2950.84447373376

In [21]:
total_estimate_agg_docs = estimated_agg_docs_per_company_df['estimate'].sum()
estimated_agg_doc_count = round(total_estimate_agg_docs / average_company_count_for_doc)
estimated_agg_org_count = round(estimated_agg_orgs_per_company_df['estimate'].sum() / average_company_count_for_org)
estimated_total_doc_count = estimated_agg_doc_count + individual_count
estimated_total_org_count = estimated_agg_org_count + individual_org_count
print('Estimated total count of HCP:', estimated_total_doc_count)
print('Estimated total count of HCO:', estimated_total_org_count)

Estimated total count of HCP: 3076.0
Estimated total count of HCO: 830.0


In [22]:
total_agg_amount = agg_df[agg_df['type'] == 'hcp']['amount'].sum()
total_ind_amount = df[df['type'] == 'hcp']['computed_total'].sum()

@interact(x=(1.0,55.0,0.2))
def estime_doc_count(x=1.2):
    count = round(total_estimate_agg_docs / x)
    print('Estimated total count of HCP: %d (%f companies per HCP)' % (count + individual_count, x))
    print('Average for named HCP: %d %s' % (total_ind_amount / individual_count, CURRENCY))
    print('Average for aggregated HCP: %d %s' % (total_agg_amount / count, CURRENCY))
    return

### 2: How many HCP are known by name?

In [23]:
individual_count

145

### 2 a) How many HCO are known by name?

In [24]:
individual_org_count

241

### 3: How many HCP of all HCPs that got money are known by name?

In [25]:
print(round(individual_count / estimated_total_doc_count * 100, 2), '%')

4.71 %


### 3a) How many HCO of all HCOs that got money are known by name?

In [26]:
print(round(individual_org_count / estimated_total_org_count * 100, 2), '%')

29.04 %


### 3b) How much HCP money went to named HCP?

In [27]:
individual_hcp_amount = df[df['type'] == 'hcp']['computed_total'].sum()
agg_hcp_amount = agg_df[agg_df['type'] == 'hcp']['amount'].sum()
print('Sum of money for named HCP: {:,.2f} {}'.format(individual_hcp_amount, CURRENCY))
print('Sum of money for aggregated HCP: {:,.2f} {}'.format(agg_hcp_amount, CURRENCY))
total_hcp_amount = individual_hcp_amount + agg_hcp_amount
print('Total: {:,.2f} {}'.format(total_hcp_amount, CURRENCY))
print('Percent named HCP amount to total:', round(individual_hcp_amount / total_hcp_amount * 100, 2), '%')

Sum of money for named HCP: 419,063.00 PLN
Sum of money for aggregated HCP: 8,574,003.68 PLN
Total: 8,993,066.68 PLN
Percent named HCP amount to total: 4.66 %


### 3c) How much HCO money went to named HCO?

In [28]:
individual_hco_amount = df[df['type'] == 'hco']['computed_total'].sum()
agg_hco_amount = agg_df[agg_df['type'] == 'hco']['amount'].sum()
print('Sum for named HCO: {:,.2f} {}'.format(individual_hco_amount, CURRENCY))
print('Sum for aggregated HCO: {:,.2f} {}'.format(agg_hco_amount, CURRENCY))
total_hco_amount = individual_hco_amount + agg_hco_amount
print('Total HCO sum: {:,.2f} {}'.format(total_hco_amount, CURRENCY))
print('Percent named HCO amount of total:', round(individual_hco_amount / total_hco_amount * 100, 2), '%')

Sum for named HCO: 7,339,682.00 PLN
Sum for aggregated HCO: 2,181,808.78 PLN
Total HCO sum: 9,521,490.78 PLN
Percent named HCO amount of total: 77.09 %


### 4: How much money have HCP gotten on average?

In [29]:
individual_total_amount_by_type = df.groupby('type')[MONEY_FIELDS_ONLY].sum().sum(1)
print('Individual Total Amount')
individual_total_amount_by_type

Individual Total Amount


type
hco   7,339,682.00
hcp     419,063.00
dtype: float64

In [30]:
agg_total_amount_by_type = agg_df.groupby('type')['amount'].sum()
print('Aggregated Total Amount')
agg_total_amount_by_type

Aggregated Total Amount


type
hco   2,181,808.78
hcp   8,574,003.68
Name: amount, dtype: float64

In [31]:
total_amount_by_type = individual_total_amount_by_type + agg_total_amount_by_type
print('Sum of Total Amounts')
total_amount_by_type

Sum of Total Amounts


type
hco   9,521,490.78
hcp   8,993,066.68
dtype: float64

In [32]:
print('Average amount per HCP', round(total_amount_by_type['hcp'] / estimated_total_doc_count), CURRENCY)

Average amount per HCP 2924.0 PLN


In [33]:
print('Average amount per HCO', round(total_amount_by_type['hco'] / estimated_total_org_count), CURRENCY)

Average amount per HCO 11472.0 PLN


### 5: How much money do named HCP get on average?

In [34]:
round(df[df['type'] == 'hcp']['computed_total'].sum() / individual_count)

2890.0

### 6: Top 10 named HCP

In [35]:
top_docs = df[df['type'] == 'hcp'].groupby(['uid'])['computed_total'].sum().sort_values(ascending=False).head(10)

for uid, amount in top_docs.iteritems():
    m = df[df['uid'] == uid].iloc[0]
    print(m['name'], 'earned', amount)

Grażyna Cholewińska-Szymańska earned 24638.0
Konrad Rejdak earned 20907.0
Stanisław Sierakowski earned 15226.0
Jacek Tadeusz Lelakowski earned 15000.0
Andrzej Szczudlik earned 14659.0
Tomasz Chorągiewicz earned 14615.0
Marek Beniowski earned 13272.0
Agnieszka Dębska-Cichońska earned 11725.0
Tomasz Pytrus earned 11528.0
Wesam Hussein Taslaq earned 10615.0


In [36]:
df[df['name'].str.contains('Grunze')][['name', 'first_name', 'address', 'location', 'computed_total', 'uid', 'company']]

,name,first_name,address,location,computed_total,uid,company


### 7: Which HCP got most in each category?

In [37]:
label_sum_per_doc = df[df['type'] == 'hcp'].groupby('uid')[MONEY_FIELDS_ONLY].sum()
for k in MONEY_FIELDS_ONLY:
    idx = label_sum_per_doc[k].idxmax()
    if pd.isnull(idx):
        continue
    doc = df[df['uid'] == label_sum_per_doc[k].idxmax()].iloc[0]
    print('{name} ({address}, {location}) got {amount} {currency} in {cat}.'.format(
            cat=k, name=doc['name'], address=doc['address'], location=doc['location'], currency=CURRENCY,
            amount=round(label_sum_per_doc[k].max(), 2)))

Anna Dusza (ul.Hoża 19, Warszawa) got 3288.0 PLN in related_expenses.
Tomasz Chorągiewicz (ul.Chmielna 1, Lublin) got 9900.0 PLN in registration_fees.
Stanisław Sierakowski (ul.Skłodowskiej-Curie 24A, Białystok) got 12756.0 PLN in travel_accommodation.
Konrad Rejdak (ul.Grażyny 18, Lublin) got 20000.0 PLN in fees.


### 8: Which pharma company paid the most to doctors

In [38]:
doc_total_by_company = df[df['type'] == 'hcp'].groupby('company')['computed_total'].sum()

In [39]:
doc_agg_total_by_company = agg_df[agg_df['type'] == 'hcp'].groupby('company')['amount'].sum()

In [40]:
doc_total_by_company = doc_total_by_company.add(doc_agg_total_by_company, fill_value=0)
doc_total_by_company = doc_total_by_company.sort_values(ascending=False).to_frame()
doc_total_by_company['rank'] = doc_total_by_company.rank(ascending=False)
doc_total_by_company

,0,rank
company,,
abbvie,"4,611,690.00",1.00
bayer,"4,381,376.68",2.00


### 8 b) Total amount of money to doctors

In [41]:
print('{0:,.2f} €'.format(
    doc_total_by_company[0].sum()
))

8,993,066.68 €


### 8 c) Amount for research


In [42]:
print('{0:,.2f} €'.format(
    company_df.groupby('company')['amount'].sum().sum()
))

24,227,227.44 €


### 8 d) Total amounts per company

In [43]:
ind_by_company = df.groupby('company')['computed_total'].sum()
agg_by_company = agg_df.groupby('company')['amount'].sum()
rnd_by_company = company_df.groupby('company')['amount'].sum()

total_with_rnd_by_company = ind_by_company.add(agg_by_company, fill_value=0).add(rnd_by_company, fill_value=0)
total_with_rnd_by_company = total_with_rnd_by_company.sort_values(ascending=False).to_frame()
total_with_rnd_by_company['rank'] = total_with_rnd_by_company.rank(ascending=False)
total_with_rnd_by_company

,0,rank
company,,
abbvie,"23,191,967.00",1.00
bayer,"19,549,817.90",2.00


In [44]:
print('{:,.2f} {}'.format(
    total_with_rnd_by_company[0].sum(), CURRENCY
))

42,741,784.90 PLN


### 9: Which pharma company has the highest payment to HCPs on average?

In [45]:
count_docs_per_company = df[df['type'] == 'hcp'].groupby(['company', 'uid']).size().unstack().sum(1)
estimated_total_docs_per_company = count_docs_per_company + estimated_agg_docs_per_company_df['estimate']
average_amount_per_doc_per_company = (doc_total_by_company[0] / estimated_total_docs_per_company).to_frame()
average_amount_per_doc_per_company = average_amount_per_doc_per_company.sort_values(0, ascending=False)
average_amount_per_doc_per_company['rank'] = average_amount_per_doc_per_company.rank(ascending=False)
average_amount_per_doc_per_company['estimated_count'] = estimated_total_docs_per_company
average_amount_per_doc_per_company

,0,rank,estimated_count
company,,,
abbvie,"3,066.86",1.00,"1,503.72"
bayer,"2,750.17",2.00,"1,593.13"


### 10: Which HCP got money from the most companies?

In [46]:
top_docs_by_company_count = top_docs_by_company_count.sort_values(ascending=False)

In [47]:
for i, count in top_docs_by_company_count.head(6).iteritems():
    matches = df[df['uid'] == i]
    company_sums = matches.groupby('company')['computed_total'].sum()
    total_sums = company_sums.sum()
    doc = matches.iloc[0]
    print(doc['name'], doc['address'], doc['location'], count)
    print(company_sums)
    print('Total', total_sums)
    print('-' * 20)

Piotr Landowski ul.Nowe Ogrody 1-6 Gdańsk 2.0
company
abbvie     258.00
bayer    1,000.00
Name: computed_total, dtype: float64
Total 1258.0
--------------------
Marcin Hawro ul.Paderewskiego 5 Łańcut 1.0
company
abbvie   554.00
Name: computed_total, dtype: float64
Total 554.0
--------------------
Jerzy Paleczny ul.Wyzwolenia 18 Bielsko-Biała 1.0
company
abbvie   3,019.00
Name: computed_total, dtype: float64
Total 3019.0
--------------------
Tomasz Chorągiewicz ul.Chmielna 1 Lublin 1.0
company
bayer   14,615.00
Name: computed_total, dtype: float64
Total 14615.0
--------------------
Arkadiusz Kazimierczak Przecław 49a/5 Przecław 1.0
company
bayer   4,501.00
Name: computed_total, dtype: float64
Total 4501.0
--------------------
Marta Wilkowska-Trojniel Ul.Stołeczna 7 l. 9 Białystok 1.0
company
bayer   1,450.00
Name: computed_total, dtype: float64
Total 1450.0
--------------------


### 11: Which pharma companies have a low rate of publication among HCPs?

In [48]:
total_docs_per_company = estimated_agg_docs_per_company_df['estimate'] + count_docs_per_company
publication_percentage = (count_docs_per_company / total_docs_per_company * 100).to_frame()
publication_percentage['published_docs'] = count_docs_per_company
publication_percentage['estimate_non_published_docs'] = estimated_agg_docs_per_company_df['estimate']
publication_percentage = publication_percentage.sort_values(0)
publication_percentage['rank'] = publication_percentage[0].rank()
publication_percentage.head(10)

,0,published_docs,estimate_non_published_docs,rank
company,,,,
bayer,4.14,66.00,"1,527.13",1.00
abbvie,5.32,80.00,"1,423.72",2.00


### 12: Which pharma companies have a high rate of publication among HCPs?

In [49]:
publication_percentage = publication_percentage.sort_values(0, ascending=False)
publication_percentage['rank'] = publication_percentage[0].rank(ascending=False)
publication_percentage.head(10)

,0,published_docs,estimate_non_published_docs,rank
company,,,,
abbvie,5.32,80.00,"1,423.72",1.00
bayer,4.14,66.00,"1,527.13",2.00


In [50]:
publication_percentage[0].mean()

4.7314724207964716

### Deciles for HCO computed total

In [51]:
df[df['type'] == 'hco']['computed_total'].quantile([x / 10 for x in range(1, 10)])

0.10    1,000.00
0.20    2,000.00
0.30    3,011.50
0.40    5,000.00
0.50    8,000.00
0.60   10,000.00
0.70   16,533.60
0.80   29,200.00
0.90   60,117.30
Name: computed_total, dtype: float64

### Gender split

(if available)

In [52]:
df.groupby('gender')['computed_total'].mean()

Series([], Name: computed_total, dtype: float64)

### Percentage of 'real' doctors 

How many have a 'Dr.' degree.

In [53]:
df['has_doc'] = df['title'].str.contains('Dr|med')
is_hcp = df['type'] == 'hcp'
doc_percent = (df[is_hcp & df['has_doc']].groupby('company').size() / df[is_hcp].groupby('company').size() * 100).fillna(0)
doc_percent

company
abbvie    0.00
bayer    42.42
dtype: float64

In [54]:
companies_with_titles = list(doc_percent.where(doc_percent > 0).dropna().index)
companies_with_titles

['bayer']

In [55]:
entries_with_possible_titles = df[df['company'].isin(companies_with_titles) & is_hcp]
len(entries_with_possible_titles)

66

In [56]:
hcps_with_possible_titles = entries_with_possible_titles.groupby('uid')
print('HCPs that can have a title attached', len(hcps_with_possible_titles))

HCPs that can have a title attached 66


In [57]:
docs_with_titles = hcps_with_possible_titles['has_doc'].any().sum()
print('HCPs that have a Dr title', docs_with_titles, ', that is ', docs_with_titles / len(hcps_with_possible_titles) * 100, '%')

HCPs that have a Dr title 28 , that is  42.4242424242 %
